In [1]:
# VGG16 with finetuning

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [3]:
# load CIFAR-10 dataset
(train_images, train_labels), (test_images,test_labels) =   datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [4]:
len(train_images), len(train_labels), len(test_images), len(test_labels),

(50000, 50000, 10000, 10000)

In [5]:
#Normalize pixel values to be between 0 and 1

In [6]:
train_images, test_images = train_images/255.0, test_images/255.0

In [7]:
# Load pretrained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58889256/58889256 [==============================] - 2s 0us/step


In [8]:
base_model

In [9]:
#Freeze convolutional layers
for layer in base_model.layers:
  layer.trainable = False

In [10]:
# Add customer classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')

  ]
)

In [11]:
# Compile the model
model.compile(optimizer=Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 22s 11ms/step - loss: 1.4903 - accuracy: 0.4746 - val_loss: 1.2876 - val_accuracy: 0.5456
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.3029 - accuracy: 0.5452 - val_loss: 1.2243 - val_accuracy: 0.5737
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2468 - accuracy: 0.5620 - val_loss: 1.1799 - val_accuracy: 0.5809
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2163 - accuracy: 0.5737 - val_loss: 1.1719 - val_accuracy: 0.5825
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1899 - accuracy: 0.5845 - val_loss: 1.1450 - val_accuracy: 0.5940
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1689 - accuracy: 0.5887 - val_loss: 1.1437 - val_accuracy: 0.5997
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1567 - accuracy: 0.5915 - val_loss: 1.1323 -

In [14]:
# Evlauate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 8ms/step - loss: 1.1279 - accuracy: 0.6080


In [15]:
print(test_loss, test_acc)

1.1278624534606934 0.6079999804496765


In [16]:
# Fine-tuning


In [17]:
#unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:
  layer.trainable = True

In [18]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 35s 20ms/step - loss: 1.1078 - accuracy: 0.6217 - val_loss: 0.9406 - val_accuracy: 0.6766
Epoch 2/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.8455 - accuracy: 0.7099 - val_loss: 0.8441 - val_accuracy: 0.7073
Epoch 3/10
1563/1563 [==============================] - 31s 20ms/step - loss: 0.7133 - accuracy: 0.7524 - val_loss: 0.7861 - val_accuracy: 0.7316
Epoch 4/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.6087 - accuracy: 0.7876 - val_loss: 0.8147 - val_accuracy: 0.7341
Epoch 5/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.5071 - accuracy: 0.8225 - val_loss: 0.8097 - val_accuracy: 0.7448
Epoch 6/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.4183 - accuracy: 0.8524 - val_loss: 0.8789 - val_accuracy: 0.7411
Epoch 7/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.3343 - accuracy: 0.8825 - val_loss: 0.9703 -

In [22]:
# try using differnet learning rates
learning_rates = [0.0001, 0.0005, 0.001, 0.005]
results = {}

In [25]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')

  ]
)

In [26]:
for lr in learning_rates:
  model.compile(optimizer=Adam(learning_rate=lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
  # Store the results
  results[lr] = history.history['val_accuracy']

Epoch 1/10
1563/1563 [==============================] - 33s 20ms/step - loss: 0.2635 - accuracy: 0.9167 - val_loss: 1.0810 - val_accuracy: 0.7472
Epoch 2/10
1563/1563 [==============================] - 30s 19ms/step - loss: 0.0954 - accuracy: 0.9694 - val_loss: 1.1703 - val_accuracy: 0.7508
Epoch 3/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0760 - accuracy: 0.9757 - val_loss: 1.2601 - val_accuracy: 0.7451
Epoch 4/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.0679 - accuracy: 0.9782 - val_loss: 1.2858 - val_accuracy: 0.7466
Epoch 5/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.0574 - accuracy: 0.9816 - val_loss: 1.4090 - val_accuracy: 0.7465
Epoch 6/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.0566 - accuracy: 0.9823 - val_loss: 1.4180 - val_accuracy: 0.7441
Epoch 7/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.0529 - accuracy: 0.9824 - val_loss: 1.3621 -

In [ ]:
for lr, val_acc in results.items():
    print(f"Learning rate: {lr}, Validation Accuracy: {max(val_acc)}")